# 2장. 지저분한 데이터 가공하기
아직도 비즈니스 현장에서는 엑셀로 사람이 수작업을 하여 만든 데이터가 많다. 이런 데이터를 가지고 분석을 하게 되면 여러 가지 오류에 맞닥뜨릴 수 있어서,  
이번에는 지저분한 데이터를 가공하는 것을 공부해 보자.  

---
대리점 데이터(관리자가 수기로 입력함)이며, 상품 A~Z까지 26개의 상품을 취급함.
- uriage.csv: 매출 이력 데이터(2019.01~2019.07)
- kokyaku_daicho.xlsx : 대리점에서 관리하는 고객정보 데이터

## Tech 11. 데이터를 읽어들이자.

In [66]:
import pandas as pd
uriage = pd.read_csv('2장/uriage.csv')
ko = pd.read_excel('2장/kokyaku_daicho.xlsx')

display(uriage.head(), ko.head())

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


데이터를 확인해보면, item_name, item_price에 결측치나 오류가 보임.  
그리고 두 데이터를 조인시켜야 하는데 두 데이터의 컬럼명도 일치하지가 않음. ko 데이터의 '등록일'은 시간도 잘못되어 있다.. 42782 시간이 어딨어?  
=> 데이터의 정합성에 오류가 있다.

## Tech 12. 데이터의 오류를 살펴보자

In [67]:
#uriage 데이터에서 item_name의 오류를 살펴보자.
uriage['item_name'].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [68]:
uriage['item_name'].value_counts()

상품R     133
상품S     133
상품A     133
상품P     132
상품C     130
       ... 
상품t       1
상 품M      1
상품 A      1
상 품F      1
상품a       1
Name: item_name, Length: 99, dtype: int64

상품은 모두 A~Z까지만 있는데, 대문자와 소문자가 섞여 있으며 공백도 들어가 있다.. 
- 상품A
- 상품 A
- 상품a 
- 상품 a  
=> 이런 것들은 모두 다 같은 '상품A'로 보아야 한다!

In [69]:
#uriage에서 item_price도 확인
uriage['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

In [70]:
uriage['item_price'].value_counts(dropna=False)

NaN       387
1600.0    126
1800.0    119
100.0     117
1900.0    116
400.0     115
300.0     113
2200.0    111
1500.0    111
600.0     107
1700.0    107
2000.0    105
1300.0    103
1400.0    103
1000.0    101
800.0     101
900.0     100
1200.0     99
2300.0     99
2400.0     99
2100.0     96
700.0      96
200.0      95
500.0      92
1100.0     92
2500.0     88
2600.0      1
Name: item_price, dtype: int64

숫자 데이터는 숫자만 들어 있으나, 결측치(NaN)이 387개나 있다. 결측치가 이렇게 많으면 분석에 방해가 되기 때문에 잘 처리를 하는 것이 필요!  
우선은 시험 삼아서 이 데이터들을 전처리하지 않고 집계해보면 어떻게 되는지를 알아보자.

## Tech 13. 데이터에 오류가 있는 상태로 집계해보자
데이터의 오류를 제거하지 않는다면 얼마나 큰 영향을 미치는지를 알아보기!

In [71]:
#먼저, uriage 데이터(매출이력)에서 월별, 상품별로 몇 개나 팔렸는지&매출합계 집계 
uriage.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [72]:
uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'])
uriage['purchase_ym'] = uriage['purchase_date'].dt.strftime("%Y%m")
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상 품 S,NaN,김우찬,201907
2,2019-05-11 19:42:00,상 품 a,NaN,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품a,NaN,김강현,201904


In [73]:
#피봇테이블 만들기 (가로 index: purchase_ym, 세로 column: item_name(상품명), 값: 팔린 개수)
uriage.pivot_table(index = 'purchase_ym', columns = 'item_name', aggfunc ='size', fill_value = 0) #size를 쓰면 NaN도 함께 집계함

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_ym,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


같은 상품인 상품 S와 상품 s가 다른 값으로 잡혀 집계된다.  
또한 여기서 상품은 A~Z까지 총 26개여야만 하는데, 컬럼 수를 보면 99개로 늘어나있다.

In [74]:
#피봇테이블 만들기 (가로 index: purchase_ym, 세로 column: item_name(상품명), 값: 매출합계)
uriage.pivot_table(index = 'purchase_ym', columns='item_name', values='item_price', aggfunc = 'sum', fill_value = 0)

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_ym,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


마찬가지로 집계결과가 옳지 않다. 이제 이러한 오류를 수정해보자.

## Tech 14. 상품명 오류 수정하기

In [75]:
#uriage 데이터에서 상품명(item_name)수정
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상 품 S,NaN,김우찬,201907
2,2019-05-11 19:42:00,상 품 a,NaN,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품a,NaN,김강현,201904


In [76]:
pd.unique(uriage.item_name)

array(['상품A', '상 품 S', '상 품 a', '상품Z', '상품a', '상품S', '상품 a', '상품V', '상품O',
       '상 품U', '상품L', '상  품V', '상 품O', '상품C', '상품I', '상품r', '상품X', '상품 g',
       '상품R', '상품P', '상품Q', '상품y', '상품 A', '상품N', '상품W', '상 품E', '상품K',
       '상품B', '상품F', '상 품s', ' 상품W', ' 상 품 n', '상 품F', '상품D', '상품M',
       '상품Y', '상품U', '상품H', '상품T', '상품J', '상  품O', '상품E', '상  품Q',
       ' 상품 S', ' 상품 M', '상  품T', '상품G', '상 품G', ' 상품 P', ' 상품 E', '상 품N',
       '상 품Y', '상품 J', '상품 V', '상품 K', '상 품V', '상 품D', '상 품A', '상품 F',
       '상품 H', '상 품K', '상 품T', '상품 X', '상품 Q', '상 품X', '상 품H', '상 품C',
       '상품 B', '상품 O', '상품 T', '상품v', '상품p', '상품i', '상품 w', '상 품 s',
       '상 품 q', '상품s', '상품l', '상품t', '상품k', '상품g', '상품o', '상품 R', '상품 S',
       '상 품M', '상품j', '상품d', '상품 I', '상품 E', '상품 o', '상품c', '상품 v', '상품e',
       '상품x', '상 품I', ' 상품 W', ' 상품 X', '상품 M', '상 품P'], dtype=object)

In [77]:
len(pd.unique(uriage.item_name))

99

In [78]:
#26개여야 하는데 99개임.
#먼저, 상품의 공백을 모두 메꾸기
uriage['item_name'] = uriage['item_name'].str.replace(" ", '')
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,NaN,김우찬,201907
2,2019-05-11 19:42:00,상품a,NaN,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품a,NaN,김강현,201904


In [79]:
#상품 알파벳을 모두 대문자로 바꾸기
uriage['item_name'] = uriage['item_name'].str.upper()
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,NaN,김우찬,201907
2,2019-05-11 19:42:00,상품A,NaN,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,NaN,김강현,201904


In [80]:
#다시 개수를 세보자
print(len(pd.unique(uriage['item_name'])))
print(pd.unique(uriage['item_name']))  #잘 수정되었다!

26
['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']


## Tech 15. 금액의 결측치를 수정하자

In [81]:
uriage.dtypes

purchase_date    datetime64[ns]
item_name                object
item_price              float64
customer_name            object
purchase_ym              object
dtype: object

In [82]:
#이번엔 uriage 데이터의 item_price에 있던 NaN들을 처리한다.
#결측치가 있는지 알아보기
uriage.isnull().any(axis=0) #item_price에만 있음

purchase_date    False
item_name        False
item_price        True
customer_name    False
purchase_ym      False
dtype: bool

In [83]:
#얼마나 있나?
uriage.isnull().sum()

purchase_date      0
item_name          0
item_price       387
customer_name      0
purchase_ym        0
dtype: int64

결측치는 수가 적으면 아예 지워버려도 된다. 또는, 평균이나 중간값으로 채우는 등 보간법을 사용해도 된다.  
그런데 이 케이스에선 **"집계 기간에 상품 단가의 변동이 없다"는 전제 조건**이 있었으므로, 같은 상품의 단가와 같은 가격으로 채워넣으면 된다.

In [84]:
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,NaN,김우찬,201907
2,2019-05-11 19:42:00,상품A,NaN,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,NaN,김강현,201904


In [97]:
for i in list(uriage['item_name'].unique()):
    print(i, '최저가: ', uriage.loc[uriage['item_name'] == i , 'item_price'].min(skipna=False), #결측치가 있으니 NaN으로 나옴. 
         ' 최고가: ', uriage.loc[uriage['item_name']==i, 'item_price'].max()) 

상품A 최저가:  nan  최고가:  100.0
상품S 최저가:  nan  최고가:  1900.0
상품Z 최저가:  2600.0  최고가:  2600.0
상품V 최저가:  nan  최고가:  2200.0
상품O 최저가:  nan  최고가:  1500.0
상품U 최저가:  nan  최고가:  2100.0
상품L 최저가:  nan  최고가:  1200.0
상품C 최저가:  nan  최고가:  300.0
상품I 최저가:  nan  최고가:  900.0
상품R 최저가:  nan  최고가:  1800.0
상품X 최저가:  nan  최고가:  2400.0
상품G 최저가:  nan  최고가:  700.0
상품P 최저가:  nan  최고가:  1600.0
상품Q 최저가:  nan  최고가:  1700.0
상품Y 최저가:  nan  최고가:  2500.0
상품N 최저가:  nan  최고가:  1400.0
상품W 최저가:  nan  최고가:  2300.0
상품E 최저가:  nan  최고가:  500.0
상품K 최저가:  nan  최고가:  1100.0
상품B 최저가:  nan  최고가:  200.0
상품F 최저가:  nan  최고가:  600.0
상품D 최저가:  nan  최고가:  400.0
상품M 최저가:  nan  최고가:  1300.0
상품H 최저가:  nan  최고가:  800.0
상품T 최저가:  nan  최고가:  2000.0
상품J 최저가:  nan  최고가:  1000.0


In [98]:
flg_is_null = uriage['item_price'].isnull() #price가 NaN인 애들만 뽑음 -> T/F로 되어있으니 인덱싱으로 True인 애들만 뽑는다!
flg_is_null

0       False
1        True
2        True
3       False
4        True
        ...  
2994    False
2995    False
2996     True
2997     True
2998    False
Name: item_price, Length: 2999, dtype: bool

In [99]:
uriage.loc[flg_is_null, 'item_name'].unique() #price가 NaN으로 비어있는 애들 -> 이 값들 하나하나에 대해서 item_price 온전한 값을 넣어준다.

array(['상품S', '상품A', '상품P', '상품N', '상품W', '상품R', '상품I', '상품L', '상품F',
       '상품O', '상품B', '상품C', '상품V', '상품Q', '상품U', '상품K', '상품T', '상품X',
       '상품E', '상품M', '상품G', '상품J', '상품D', '상품H', '상품Y'], dtype=object)

In [100]:
uriage.loc[~flg_is_null, 'item_name'].unique() #반대로 price값이 있는 애들 

array(['상품A', '상품Z', '상품S', '상품V', '상품O', '상품U', '상품L', '상품C', '상품I',
       '상품R', '상품X', '상품G', '상품P', '상품Q', '상품Y', '상품W', '상품E', '상품K',
       '상품B', '상품F', '상품N', '상품D', '상품M', '상품H', '상품T', '상품J'],
      dtype=object)

In [101]:
uriage['item_price'].loc[(flg_is_null) & (uriage['item_name'] == '상품S')]

1      NaN
95     NaN
259    NaN
372    NaN
438    NaN
714    NaN
892    NaN
932    NaN
1044   NaN
1366   NaN
1391   NaN
1457   NaN
1551   NaN
1560   NaN
1627   NaN
2000   NaN
2283   NaN
2346   NaN
2429   NaN
2458   NaN
2489   NaN
2805   NaN
2856   NaN
2946   NaN
Name: item_price, dtype: float64

In [102]:
for i in list(uriage.loc[flg_is_null, 'item_name'].unique()):
    price = uriage.loc[(~flg_is_null) & (uriage['item_name'] == i), 'item_price'].max() #price값이 있는 애들의 값을 가져옴 
    uriage['item_price'].loc[(flg_is_null) & (uriage['item_name'] == i)] = price #그래서 비어있는 애들에 price로 채워줌 
    
uriage.head()

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [104]:
#위랑 같은 결과이긴 한데, 3번째 줄과 같이 loc를 쓸땐 행, 열 둘 다 입력해주는 게 에러 안나고 더 좋다.
for i in list(uriage['item_name'].unique()):
    price = uriage.loc[(~flg_is_null) & (uriage['item_name'] == i), 'item_price'].max() #price값이 있는 애들의 값을 가져옴 
    uriage.loc[(flg_is_null) & (uriage['item_name'] == i), 'item_price'] = price #그래서 비어있는 애들에 price로 채워줌 
    
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [105]:
#뽑아보면 결측치가 없어보이지만 혹시나 있을 수 있으니 꼭 전체 데이터 검증하기!
uriage.isnull().any(axis=0) #item_price 열 결측치 없어짐

purchase_date    False
item_name        False
item_price       False
customer_name    False
purchase_ym      False
dtype: bool

In [106]:
uriage['item_name'].sort_values().unique() #알파벳순으로 상품정렬 

array(['상품A', '상품B', '상품C', '상품D', '상품E', '상품F', '상품G', '상품H', '상품I',
       '상품J', '상품K', '상품L', '상품M', '상품N', '상품O', '상품P', '상품Q', '상품R',
       '상품S', '상품T', '상품U', '상품V', '상품W', '상품X', '상품Y', '상품Z'],
      dtype=object)

In [107]:
#이번엔 각 상품의 금액이 정상적으로 수정되었는지 확인
for i in list(uriage['item_name'].sort_values().unique()):
    print(i)
    print(uriage.loc[(uriage['item_name']==i), 'item_price'].max())
    print(uriage.loc[(uriage['item_name']==i), 'item_price'].min())
    print( )

상품A
100.0
100.0

상품B
200.0
200.0

상품C
300.0
300.0

상품D
400.0
400.0

상품E
500.0
500.0

상품F
600.0
600.0

상품G
700.0
700.0

상품H
800.0
800.0

상품I
900.0
900.0

상품J
1000.0
1000.0

상품K
1100.0
1100.0

상품L
1200.0
1200.0

상품M
1300.0
1300.0

상품N
1400.0
1400.0

상품O
1500.0
1500.0

상품P
1600.0
1600.0

상품Q
1700.0
1700.0

상품R
1800.0
1800.0

상품S
1900.0
1900.0

상품T
2000.0
2000.0

상품U
2100.0
2100.0

상품V
2200.0
2200.0

상품W
2300.0
2300.0

상품X
2400.0
2400.0

상품Y
2500.0
2500.0

상품Z
2600.0
2600.0



In [108]:
for i in list(uriage['item_name'].sort_values().unique()):
    print(i, '최저가: ', uriage.loc[uriage['item_name'] == i , 'item_price'].min(skipna=False),
         ' 최고가: ', uriage.loc[uriage['item_name']==i, 'item_price'].max())

상품A 최저가:  100.0  최고가:  100.0
상품B 최저가:  200.0  최고가:  200.0
상품C 최저가:  300.0  최고가:  300.0
상품D 최저가:  400.0  최고가:  400.0
상품E 최저가:  500.0  최고가:  500.0
상품F 최저가:  600.0  최고가:  600.0
상품G 최저가:  700.0  최고가:  700.0
상품H 최저가:  800.0  최고가:  800.0
상품I 최저가:  900.0  최고가:  900.0
상품J 최저가:  1000.0  최고가:  1000.0
상품K 최저가:  1100.0  최고가:  1100.0
상품L 최저가:  1200.0  최고가:  1200.0
상품M 최저가:  1300.0  최고가:  1300.0
상품N 최저가:  1400.0  최고가:  1400.0
상품O 최저가:  1500.0  최고가:  1500.0
상품P 최저가:  1600.0  최고가:  1600.0
상품Q 최저가:  1700.0  최고가:  1700.0
상품R 최저가:  1800.0  최고가:  1800.0
상품S 최저가:  1900.0  최고가:  1900.0
상품T 최저가:  2000.0  최고가:  2000.0
상품U 최저가:  2100.0  최고가:  2100.0
상품V 최저가:  2200.0  최고가:  2200.0
상품W 최저가:  2300.0  최고가:  2300.0
상품X 최저가:  2400.0  최고가:  2400.0
상품Y 최저가:  2500.0  최고가:  2500.0
상품Z 최저가:  2600.0  최고가:  2600.0


In [30]:
#좀더 보기좋게
for i in list(uriage['item_name'].sort_values().unique()):
    print( i +'의 최고가: ' + str(uriage.loc[(uriage['item_name']==i), 'item_price'].max()) + ', ' + '최저가: ' +str(uriage.loc[(uriage['item_name']==i), 'item_price'].min(skipna=False)))

상품A의 최고가: 100.0, 최저가: 100.0
상품B의 최고가: 200.0, 최저가: 200.0
상품C의 최고가: 300.0, 최저가: 300.0
상품D의 최고가: 400.0, 최저가: 400.0
상품E의 최고가: 500.0, 최저가: 500.0
상품F의 최고가: 600.0, 최저가: 600.0
상품G의 최고가: 700.0, 최저가: 700.0
상품H의 최고가: 800.0, 최저가: 800.0
상품I의 최고가: 900.0, 최저가: 900.0
상품J의 최고가: 1000.0, 최저가: 1000.0
상품K의 최고가: 1100.0, 최저가: 1100.0
상품L의 최고가: 1200.0, 최저가: 1200.0
상품M의 최고가: 1300.0, 최저가: 1300.0
상품N의 최고가: 1400.0, 최저가: 1400.0
상품O의 최고가: 1500.0, 최저가: 1500.0
상품P의 최고가: 1600.0, 최저가: 1600.0
상품Q의 최고가: 1700.0, 최저가: 1700.0
상품R의 최고가: 1800.0, 최저가: 1800.0
상품S의 최고가: 1900.0, 최저가: 1900.0
상품T의 최고가: 2000.0, 최저가: 2000.0
상품U의 최고가: 2100.0, 최저가: 2100.0
상품V의 최고가: 2200.0, 최저가: 2200.0
상품W의 최고가: 2300.0, 최저가: 2300.0
상품X의 최고가: 2400.0, 최저가: 2400.0
상품Y의 최고가: 2500.0, 최저가: 2500.0
상품Z의 최고가: 2600.0, 최저가: 2600.0


최대금액과 최저금액이 일치하므로, 제대로 전처리했음을 확인!  
skipna의 경우 True가 디폴트(NaN을 무시하고 집계), 여기선 False로 했으므로 NaN이 있는 경우는 최저가가 NaN으로 나오게 된다. 여기선 NaN이 없으므로 안나온다.

## Tech 16. 고객 이름의 오류를 수정하자
이번엔 ko 데이터!

In [109]:
ko.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


In [110]:
ko['고객이름'].sort_values()

141    김 가온
75     김 강민
135    김 강현
132    김 건우
116    김 건희
       ... 
7      김 호준
133    정 도형
122    정 영훈
152    정 우석
173    정 준기
Name: 고객이름, Length: 200, dtype: object

In [111]:
uriage['customer_name'].sort_values()

0       김가온
2282    김가온
495     김가온
568     김가온
2388    김가온
       ... 
1534    정준기
1092    정준기
1365    정준기
2538    정준기
2998    정준기
Name: customer_name, Length: 2999, dtype: object

두 데이터를 결합하려면 ko 데이터의 '고객이름' 값과, uriage 데이터의 'customer_name' 값이 같아야 하는데 현재 서식이 안 맞다.  
ko 데이터의 값에 공백이 있어 이를 제거하자.

In [112]:
ko['고객이름'] = ko['고객이름'].str.replace("  ","")
ko['고객이름'] = ko['고객이름'].str.replace(" ",'')
ko['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

## Tech 17. 날짜 오류를 수정하자

In [113]:
#ko 데이터를 잘 보면.. 등록일에 이상한 값이 있다. 42782..??
ko.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04 00:00:00
1,김도윤,E시,42782
2,김지한,A시,2018-01-07 00:00:00
3,김하윤,F시,42872
4,김시온,E시,43127


In [114]:
print(ko.shape)

(200, 3)


이는 엑셀 데이터를 다룰 때 주의할 점으로, 서식이 다른 데이터가 섞여 있을 수 있다는 것!  
이번엔 42782같은 정체불명의 숫자를 날짜로 바꿔 보자.

In [115]:
ko.dtypes

고객이름    object
지역      object
등록일     object
dtype: object

In [117]:
ko['등록일'].astype('str').str.isdigit() #위처럼 42782 같은 것은 숫자로 인식됨 

0      False
1       True
2      False
3       True
4       True
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 등록일, Length: 200, dtype: bool

In [118]:
ko['등록일'].astype('str').str.isdigit().sum() #이런 것들의 개수는 22개 

22

In [40]:
#전략: 위에서 숫자로 판명된 것(True)만 인덱싱 후, timedelta 메서드를 사용해 날짜로 바꾼 후, 1900-01-01 날짜를 기준으로 이 시간만큼 더하기 
is_serial = ko['등록일'].astype('str').str.isdigit()

In [41]:
pd.to_timedelta(ko.loc[is_serial, '등록일'].astype('float')-2, unit='D')

1     42780 days
3     42870 days
4     43125 days
21    42918 days
27    42899 days
47    42739 days
49    42927 days
53    42831 days
76    43186 days
80    43108 days
99    42883 days
114   43252 days
118   43127 days
122   43204 days
139   42878 days
143   42816 days
155   42752 days
172   43179 days
179   42741 days
183   42938 days
186   43292 days
192   43257 days
Name: 등록일, dtype: timedelta64[ns]

In [42]:
from_serial = pd.to_timedelta(ko.loc[is_serial, '등록일'].astype('float')-2, unit='D') + pd.to_datetime("1900/01/01")
from_serial

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 등록일, dtype: datetime64[ns]

In [43]:
#앞서 숫자로 판명되지 않은 것(False)도 datetime으로 바꿔주기
from_string = pd.to_datetime(ko.loc[~is_serial,'등록일'])
from_string

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [44]:
#from_serial, from_string을 합치고, ko 데이터셋의 '등록일' 컬럼으로 넣어줌 
pd.concat([from_serial, from_string])

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 200, dtype: datetime64[ns]

In [45]:
#위를 보면 인덱스가 제대로 안 맞아 있는데(concat은 그냥 냅다 세로로 붙이는 거니까), 데이터프레임에 넣으면 알아서 인덱스에 맞춰서 들어가진다.
ko['등록일'] = pd.concat([from_serial, from_string])
ko.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-16
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-17
4,김시온,E시,2018-01-27


In [46]:
ko.dtypes

고객이름            object
지역              object
등록일     datetime64[ns]
dtype: object

In [47]:
#등록일로부터 등록연월 추출하기
ko['등록연월'] = ko['등록일'].dt.strftime("%Y%m")
ko.head()

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-16,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-17,201705
4,김시온,E시,2018-01-27,201801


In [48]:
#등록연월별 고객 수 카운트 (count는 NaN을 빼고 셈. ko 데이터는 NaN이 없으니 괜찮다)
ko.groupby('등록연월')['고객이름'].count()

등록연월
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

In [49]:
print(ko.shape) #shape 확인해보면 행 개수도 전처리 전과 같다.

(200, 4)


In [50]:
#등록일 컬럼에 숫자가 남아있을까? -> No! 전처리 잘 됨 
ko['등록일'].astype('str').str.isdigit().sum()

0

## Tech 18. 고객이름을 key로 두 데이터를 조인하자.

In [51]:
display(uriage.head(), ko.head())

,purchase_date,item_name,item_price,customer_name,purchase_ym
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-16,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-17,201705
4,김시온,E시,2018-01-27,201801


In [52]:
#고객이름을 key로 하여, 두 데이터를 조인 -> uriage를 기준으로 left join(구매내역이 있는 것만 가져올 거라서)
join_data = pd.merge(uriage, ko, left_on = 'customer_name', right_on = '고객이름', how='left')
join_data

,purchase_date,item_name,item_price,customer_name,purchase_ym,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,김정민,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,김재원,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,김지율,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,김승주,201907,김승주,E시,2018-07-13,201807


## Tech 19. 정제한 데이터를 덤프하자
여기까지 만든 데이터에서 끝내면 나중에 분석할 때 이 과정을 처음부터 다시 해야 한다.  
이를 방지하기 위해, 만든 데이터를 새로운 파일로 저장(덤프)하자!

In [53]:
#위의 join_data의 칼럼 위치 수정, 필요한 것만 골라오기 (나중에 다루기 좋게)
dump_data = join_data[['purchase_date', 'purchase_ym', 'item_name', 'item_price', '고객이름', '지역', '등록일']]
dump_data

,purchase_date,purchase_ym,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-13


In [54]:
#csv 파일로 다운로드
dump_data.to_csv('dump_data.csv', index=False) #index=False를 안넣으면 인덱스가 새로운 칼럼으로 들어가기 때문에 꼭 써주자.

다 한 뒤에, 저장 위치에 csv 파일이 잘 저장되었는지를 확인하기!

## Tech 20. 데이터를 집계하자 
앞서 만든 dump data를 불러와서 데이터 분석을 간단히 해보자.

In [55]:
import pandas as pd
data = pd.read_csv('dump_data.csv')
data.head()

,purchase_date,purchase_ym,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07 00:00:00


In [56]:
#purchase_ym을 인덱스로 하여 상품별 팔린 개수 집계 (index:purchase_ym, column:item_name)
data.pivot_table(index = 'purchase_ym', columns='item_name', aggfunc = 'size', fill_value = 0)

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_ym,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [57]:
#이번엔 월별, 상품별 매출금액 집계
data.pivot_table(index = 'purchase_ym', columns = 'item_name', values = 'item_price', aggfunc='sum', fill_value = 0)

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_ym,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [58]:
#월별 고객별 구매횟수 
data.pivot_table(index = 'purchase_ym', columns = '고객이름', aggfunc = 'size', fill_value = 0)

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_ym,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,...,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,...,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,...,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,...,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,...,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,...,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,...,2,1,0,4,0,2,6,1,2,3


In [59]:
#월별 지역별 구매횟수
data.pivot_table(index = 'purchase_ym', columns = '지역' , aggfunc = 'size' , fill_value = 0)

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_ym,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [60]:
#마지막으로, 집계기간에 구매이력이 없는 사용자를 살펴보자.
#이걸 보려면 uriage는 구매데이터고, ko는 전체사용자 데이터이므로 -> ko를 기준으로 right join
away = pd.merge(uriage, ko, left_on = 'customer_name', right_on = '고객이름', how='right')
away

,purchase_date,item_name,item_price,customer_name,purchase_ym,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906,김가온,C시,2017-01-26,201701
1,2019-06-14 09:08:00,상품A,100.0,김가온,201906,김가온,C시,2017-01-26,201701
2,2019-01-30 06:49:00,상품J,1000.0,김가온,201901,김가온,C시,2017-01-26,201701
3,2019-07-30 09:17:00,상품X,2400.0,김가온,201907,김가온,C시,2017-01-26,201701
4,2019-04-07 09:37:00,상품A,100.0,김가온,201904,김가온,C시,2017-01-26,201701
...,...,...,...,...,...,...,...,...,...
2995,2019-04-14 04:16:00,상품L,1200.0,김현진,201904,김현진,H시,2017-07-13,201707
2996,2019-06-24 02:56:00,상품A,100.0,김현진,201906,김현진,H시,2017-07-13,201707
2997,2019-06-30 18:30:00,상품X,2400.0,김현진,201906,김현진,H시,2017-07-13,201707
2998,2019-03-06 22:30:00,상품X,2400.0,김현진,201903,김현진,H시,2017-07-13,201707


In [61]:
#위를 보면 마지막 행처럼 사용자정보는 있는데 구매이력은 없는 애들이 있음. 이런 데이터가 몇개인지가 궁금!!
#구매이력이 없으면 purchase_date가 비어있을 것이므로..
away['purchase_date'].isnull().sum() #1개 있음

1

In [62]:
#이 사람이 누군지 고객명과 등록일만 뽑자
away[away['purchase_date'].isnull()][['고객이름','등록일']] #T/F로 되어있는 코드로 인덱싱 

,고객이름,등록일
2999,김서우,2019-04-23


In [63]:
away[away['purchase_date'].isnull()]

,purchase_date,item_name,item_price,customer_name,purchase_ym,고객이름,지역,등록일,등록연월
2999,NaT,NaN,NaN,NaN,NaN,김서우,D시,2019-04-23,201904


In [64]:
away['purchase_date'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2995    False
2996    False
2997    False
2998    False
2999     True
Name: purchase_date, Length: 3000, dtype: bool

이로써 1장, 2장에선 데이터 분석에 있어 가장 중요한 `데이터 가공(전처리)` 를 배웠음.  
이러한 전처리는 **정확성** 이 핵심!! 소홀히 하면 나중에 큰코다칠 수 있으니 꼭 정확하게! 그리고 검산도 반드시 해볼 것!  
다음 3장부터는 구체적인 분석 start.